This is the IOHMM model with the parameters learned in a semi-supervised way. By using some labeled data, we force the learning process in a certain direction. The unlabeled data will be estimated using EM algorithm iteratively. See notes in http://pages.cs.wisc.edu/~jerryzhu/pub/sslicml07.pdf

# Example use of SemiSupervised_IOHMM 

In [1]:
from __future__ import  division
import numpy as np
from copy import deepcopy
import sys
sys.path.append('./main')
sys.path.append('./auxiliary')
from SemiSupervised_IOHMM import *
import pandas as pd
import warnings
warnings.simplefilter("ignore")

## Speed data

### The sequence data

In [2]:
speed = pd.read_csv('data/speed.csv')
print speed.head()

   Unnamed: 0        rt corr  Pacc prev
0           1  6.456770  cor   0.0  inc
1           2  5.602119  cor   0.0  cor
2           3  6.253829  inc   0.0  cor
3           4  5.451038  inc   0.0  inc
4           5  5.872118  inc   0.0  inc


### The labeled states

In our structure of the code, the states should be a dictionary, the key is the index in the sequence (e.g. 0, 5) and the value is a one-out-of-n code of array where the kth value is 1 if the hidden state is k. n is the number of states in total.

In the following example, we assume that the "corr" column gives the correct hidden states. Here we assume only the first half of the sequence is labeled.

In [12]:
speed.shape[0]

439

In [30]:
states = {}
corr = np.array(speed['corr'])
for i in range(int(len(corr)/2)):
    state = np.zeros((3,))
    if corr[i] == 'cor':
        states[i] = np.array([0,1,0,0])
        speed.set_value(i, 'rt', 1)
    else:
        states[i] = np.array([1,0,0,0])
        speed.set_value(i, 'rt', 0)

In [31]:
len(states)

219

In [32]:
speed

,Unnamed: 0,rt,corr,Pacc,prev
0,1,1.000000,cor,0.000000,inc
1,2,1.000000,cor,0.000000,cor
2,3,0.000000,inc,0.000000,cor
3,4,0.000000,inc,0.000000,inc
4,5,0.000000,inc,0.000000,inc
5,6,1.000000,cor,0.000000,inc
6,7,1.000000,cor,0.045455,cor
7,8,1.000000,cor,0.090909,cor
8,9,0.000000,inc,0.136364,cor
9,10,1.000000,cor,0.181818,inc


## Setting up the model

In [44]:
SHMM = SemiSupervisedIOHMM(num_states=4)
SHMM.setData([[speed, states]])
SHMM.setModels(model_emissions = [LM()], model_transition=MNLP(solver='lbfgs'))
SHMM.setInputs(covariates_initial = [], covariates_transition = [], covariates_emissions = [[]])
SHMM.setOutputs([['rt']])

## Start training

In [45]:
SHMM.train()

-172.801996012
-171.195098278
-166.501396206
-169.546971192
-165.591560933
-165.245834333
-170.164585511
-171.603596056
-172.70730488
-172.484259641
-171.86664268
-173.754708929
-174.043988484
-174.726975439
-172.862252356
-172.125495861
-173.248786404
-174.815562112
-174.362523014
-173.075921155
-172.819022358
-172.305181246
-171.251454181
-172.478674837
-174.179394338
-172.352963174
-171.175469224
-172.336644197
-174.115772621
-172.492170605
-171.052263357
-172.097804756
-172.613665313
-171.616126787
-169.924865871
-169.950706363
-169.958657831
-170.04171893
-170.117626352
-170.165246568
-170.200268459
-170.278863811
-167.204446322
-167.894667924
-164.109390061
-161.589656051
-160.536206529
-159.979200973
-158.632290626
-125.808027873
-84.1749722814
-68.3996714286
-62.6978371673
-62.1113030028
-60.9883433837
-52.0391131871
-44.6233729502
-44.3602348673
-44.4203435438
-44.4989289936
-44.5566123577
-44.5748014147
-44.5785720634
-44.5793771472
-44.5795488142
-44.5795854001


## See coefficients

In [46]:
print np.exp(SHMM.model_transition[0].coef - logsumexp(SHMM.model_transition[0].coef))
print np.exp(SHMM.model_transition[1].coef - logsumexp(SHMM.model_transition[1].coef))

[[ 0.42176108  0.5332247   0.02250711  0.02250711]]
[[ 0.15113613  0.81203853  0.01918793  0.01763742]]


In [47]:
print SHMM.model_emissions[0][0].coef
print SHMM.model_emissions[1][0].coef
print SHMM.model_emissions[2][0].coef
print SHMM.model_emissions[3][0].coef

[-0.]
[ 1.]
[ 5.47277148]
[ 6.39095146]


In [48]:
print np.sqrt(SHMM.model_emissions[0][0].dispersion)
print np.sqrt(SHMM.model_emissions[1][0].dispersion)
print np.sqrt(SHMM.model_emissions[2][0].dispersion)
print np.sqrt(SHMM.model_emissions[3][0].dispersion)

0.0
0.0
0.18225006545
0.224628384718
